### Install Dependencies

In [5]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Import Wikipedia Page

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
tbody = soup.find('table').tbody
array = []
for tr in tbody.find_all('tr'):
    sub_array = []
    for th in tr.find_all('th'):
        sub_array.append(th.text.replace('\n', ''))
    for td in tr.find_all('td'):
        sub_array.append(td.text.replace('\n', ''))
    array.append(sub_array)
np_array = np.array(array)
df = pd.DataFrame(np_array)
df.columns = df.iloc[0]
df = df[1:]
df = df[df.Borough != 'Not assigned']
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],
    axis=1
)
df.shape

(103, 3)

In [7]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
merged_left = pd.merge(left=df,right=df_geo, how='left', left_on='Postcode', right_on='Postal Code')
merged_left.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
